# Imports

In [1]:
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, auc
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import keras
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import LinearSVC
import pandas as pd
from sklearn.model_selection import train_test_split
from time import time
import numpy as np
from cuml.svm import LinearSVC, SVC
from sklearn.svm import SVC
import cudf

2024-02-29 19:50:24.489578: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 19:50:24.489620: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 19:50:24.492301: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-29 19:50:24.503754: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-29 19:50:25.549237: W tensorflow/compiler/tf2

# Loading Data

In [2]:
data = pd.read_csv("balanced_data.csv")
data.head(), data.shape

(   Unnamed: 0  case_id  bankacctype_710L  cardtype_51L  clientscnt_136L  \
 0       14670        0                 1             2                5   
 1      637908  1602978                 1             2                5   
 2      266413  1602977                 1             2                5   
 3      842720  1602976                 1             2                5   
 4      421631  1602975                 1             2                5   
 
    credtype_322L  datefirstoffer_1144D  datelastinstal40dpd_247D  \
 0              3                  1404                      5175   
 1              3                  1404                      5175   
 2              3                  1404                      5175   
 3              3                  1404                      5175   
 4              3                  1404                      5175   
 
    datelastunpaid_3546854D  disbursementtype_67L  ...  \
 0                     5231                     3  ...   
 1        

In [3]:
data.drop("Unnamed: 0", axis = 1, inplace=True)
data.head()

,case_id,bankacctype_710L,cardtype_51L,clientscnt_136L,credtype_322L,datefirstoffer_1144D,datelastinstal40dpd_247D,datelastunpaid_3546854D,disbursementtype_67L,dtlastpmtallstes_4499206D,...,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A
0,0,1,2,5,3,1404,5175,5231,3,4464,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0
1,1602978,1,2,5,3,1404,5175,5231,3,4464,...,0.0,0.0,15998.0,0.0,2.0,0.0,0.0,0.0,35865.344,0.0
2,1602977,1,2,5,3,1404,5175,5231,3,4464,...,0.0,1.0,50000.0,1.0,4.0,27662.8,27662.8,27662.8,60912.800,22775.6
3,1602976,1,2,5,3,1404,5175,5231,3,4464,...,0.0,0.0,40650.0,0.0,1.0,0.0,0.0,0.0,88454.600,0.0
4,1602975,1,2,5,3,1404,5175,5231,3,4464,...,0.0,0.0,23998.0,0.0,0.0,0.0,0.0,0.0,87624.980,0.0


In [4]:
# data = data.sample(n = 10000)
# data.shape

In [5]:
data['target'].value_counts()

0    47994
1    47994
Name: target, dtype: int64

# Splitting the dataset

In [6]:
x = data.drop(['target'], axis = 1)
y = data['target']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

# SVM

In [7]:
x_train1, y_train1, x_test1, y_test1 = cudf.DataFrame(x_train), cudf.Series(y_train), cudf.DataFrame(x_test), cudf.Series(y_test)
modelSVC = SVC()
start = time()
modelSVC.fit(x_train1, y_train1)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_SVC = modelSVC.predict(x_test1)
acc = accuracy_score(pred_SVC , y_test1)
cm = confusion_matrix(pred_SVC , y_test1)
cr = classification_report(pred_SVC , y_test1)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

TypeError: Implicit conversion to a host NumPy array via __array__ is not allowed, To explicitly construct a GPU matrix, consider using .to_cupy()
To explicitly construct a host matrix, consider using .to_numpy().

# LGBM

In [22]:
modelLGBM = LGBMClassifier()
start = time()
modelLGBM.fit(x_train , y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_LGBM = modelLGBM.predict(x_test)
acc = accuracy_score(pred_LGBM , y_test)
cm = confusion_matrix(pred_LGBM , y_test)
cr = classification_report(pred_LGBM , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

[LightGBM] [Info] Number of positive: 3715, number of negative: 3785
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004216 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 20238
[LightGBM] [Info] Number of data points in the train set: 7500, number of used features: 186
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.495333 -> initscore=-0.018667
[LightGBM] [Info] Start training from score -0.018667
--------------------------------------------------------
Time taken: 0.7596657276153564 seconds
--------------------------------------------------------
0.976
--------------------------------------------------------
[[1256   56]
 [   4 1184]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      0.96      0.98      1312
           1       0.95      1.00 

# RF

In [23]:
model_RF = RandomForestClassifier(n_jobs=-1)
start = time()
model_RF.fit(x_train, y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_RF = model_RF.predict(x_test)
acc = accuracy_score(pred_RF , y_test)
cm = confusion_matrix(pred_RF , y_test)
cr = classification_report(pred_RF , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

--------------------------------------------------------
Time taken: 0.39954638481140137 seconds
--------------------------------------------------------
0.9664
--------------------------------------------------------
[[1249   73]
 [  11 1167]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.99      0.94      0.97      1322
           1       0.94      0.99      0.97      1178

    accuracy                           0.97      2500
   macro avg       0.97      0.97      0.97      2500
weighted avg       0.97      0.97      0.97      2500

--------------------------------------------------------


# ANN

In [47]:
# x_train2, y_train2, x_test2, y_test2 = x_train[:], y_train[:], x_test[:], y_test[:]
# y_train2 = tf.one_hot(y_train2, depth = 2)
# y_test2 = tf.one_hot(y_test2, depth = 2)

In [48]:
# x_train2.shape

In [24]:
model = keras.Sequential([
    keras.layers.Dense(256, activation='relu', input_shape = (304,)),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(), loss=tf.keras.losses.BinaryCrossentropy(), metrics=keras.metrics.CategoricalAccuracy())

start = time()
history = model.fit(x_train, y_train, epochs=5, batch_size=32, use_multiprocessing=True, validation_split=0.5)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_ANN = model.predict(x_test)

2024-02-29 17:52:19.777775: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-29 17:52:19.927227: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-29 17:52:19.927549: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/5


2024-02-29 17:52:21.524940: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-29 17:52:22.646474: I external/local_xla/xla/service/service.cc:168] XLA service 0x7efefc468590 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-29 17:52:22.646514: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2024-02-29 17:52:22.666716: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-29 17:52:22.710693: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1709209342.815310   15983 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


118/118 [==============================] - 4s 6ms/step - loss: 1701865.6250 - categorical_accuracy: 1.0000 - val_loss: 530365.2500 - val_categorical_accuracy: 1.0000
Epoch 2/5
118/118 [==============================] - 1s 5ms/step - loss: 361926.4062 - categorical_accuracy: 1.0000 - val_loss: 143587.1250 - val_categorical_accuracy: 1.0000
Epoch 3/5
118/118 [==============================] - 1s 5ms/step - loss: 35629.1992 - categorical_accuracy: 1.0000 - val_loss: 176.0419 - val_categorical_accuracy: 1.0000
Epoch 4/5
118/118 [==============================] - 1s 5ms/step - loss: 258.8718 - categorical_accuracy: 1.0000 - val_loss: 108.7242 - val_categorical_accuracy: 1.0000
Epoch 5/5
118/118 [==============================] - 1s 5ms/step - loss: 52.7885 - categorical_accuracy: 1.0000 - val_loss: 82.3906 - val_categorical_accuracy: 1.0000
--------------------------------------------------------
Time taken: 5.975755453109741 seconds
--------------------------------------------------------


In [25]:
pred = []
for i in pred_ANN:
    pred.append(np.round(i[0]))

acc = accuracy_score(pred , y_test)
cm = confusion_matrix(pred , y_test)
cr = classification_report(pred , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

0.6496
--------------------------------------------------------
[[807 423]
 [453 817]]
--------------------------------------------------------
              precision    recall  f1-score   support

         0.0       0.64      0.66      0.65      1230
         1.0       0.66      0.64      0.65      1270

    accuracy                           0.65      2500
   macro avg       0.65      0.65      0.65      2500
weighted avg       0.65      0.65      0.65      2500

--------------------------------------------------------


# LR

In [26]:
model_lr = LogisticRegression(n_jobs = -1, max_iter=1000)
start = time()
model_lr.fit(x_train, y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_lr = model_lr.predict(x_test)
acc = accuracy_score(pred_lr , y_test)
cm = confusion_matrix(pred_lr , y_test)
cr = classification_report(pred_lr , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

--------------------------------------------------------
Time taken: 5.233440637588501 seconds
--------------------------------------------------------
0.7964
--------------------------------------------------------
[[1050  299]
 [ 210  941]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.83      0.78      0.80      1349
           1       0.76      0.82      0.79      1151

    accuracy                           0.80      2500
   macro avg       0.80      0.80      0.80      2500
weighted avg       0.80      0.80      0.80      2500

--------------------------------------------------------


/home/newbie/miniconda3/envs/gpu/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# DT

In [27]:
model_DT = DecisionTreeClassifier()
start = time()
model_DT.fit(x_train, y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_DT = model_DT.predict(x_test)
acc = accuracy_score(pred_DT , y_test)
cm = confusion_matrix(pred_DT , y_test)
cr = classification_report(pred_DT , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

--------------------------------------------------------
Time taken: 0.47833943367004395 seconds
--------------------------------------------------------
0.9568
--------------------------------------------------------
[[1195   43]
 [  65 1197]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.95      0.97      0.96      1238
           1       0.97      0.95      0.96      1262

    accuracy                           0.96      2500
   macro avg       0.96      0.96      0.96      2500
weighted avg       0.96      0.96      0.96      2500

--------------------------------------------------------


# KNN

In [28]:
model_KN = KNeighborsClassifier()
start = time()
model_KN.fit(x,y)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_KN = model_KN.predict(x_test)
acc = accuracy_score(pred_KN , y_test)
cm = confusion_matrix(pred_KN , y_test)
cr = classification_report(pred_KN , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

--------------------------------------------------------
Time taken: 0.029924869537353516 seconds
--------------------------------------------------------
0.928
--------------------------------------------------------
[[1244  164]
 [  16 1076]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.99      0.88      0.93      1408
           1       0.87      0.99      0.92      1092

    accuracy                           0.93      2500
   macro avg       0.93      0.93      0.93      2500
weighted avg       0.94      0.93      0.93      2500

--------------------------------------------------------


# NB

In [29]:
model_NB = KNeighborsClassifier()
start = time()
model_NB.fit(x,y)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_NB = model_NB.predict(x_test)
acc = accuracy_score(pred_NB , y_test)
cm = confusion_matrix(pred_NB , y_test)
cr = classification_report(pred_NB , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

--------------------------------------------------------
Time taken: 0.026735305786132812 seconds
--------------------------------------------------------
0.928
--------------------------------------------------------
[[1244  164]
 [  16 1076]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.99      0.88      0.93      1408
           1       0.87      0.99      0.92      1092

    accuracy                           0.93      2500
   macro avg       0.93      0.93      0.93      2500
weighted avg       0.94      0.93      0.93      2500

--------------------------------------------------------


# MLP

In [30]:
model_NN = MLPClassifier()
start = time()
model_NN.fit(x,y)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_NN = model_NN.predict(x_test)
acc = accuracy_score(pred_NN , y_test)
cm = confusion_matrix(pred_NN , y_test)
cr = classification_report(pred_NN , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

--------------------------------------------------------
Time taken: 11.62921690940857 seconds
--------------------------------------------------------
0.8484
--------------------------------------------------------
[[1010  129]
 [ 250 1111]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.80      0.89      0.84      1139
           1       0.90      0.82      0.85      1361

    accuracy                           0.85      2500
   macro avg       0.85      0.85      0.85      2500
weighted avg       0.85      0.85      0.85      2500

--------------------------------------------------------


# XGB

In [33]:
model_XG = xgb.XGBClassifier()
start = time()
model_XG.fit(x_train,y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_XG = model_XG.predict(x_test)
acc = accuracy_score(pred_XG , y_test)
cm = confusion_matrix(pred_XG , y_test)
cr = classification_report(pred_XG , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

--------------------------------------------------------
Time taken: 0.9836869239807129 seconds
--------------------------------------------------------
0.9736
--------------------------------------------------------
[[1251   57]
 [   9 1183]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.99      0.96      0.97      1308
           1       0.95      0.99      0.97      1192

    accuracy                           0.97      2500
   macro avg       0.97      0.97      0.97      2500
weighted avg       0.97      0.97      0.97      2500

--------------------------------------------------------


# AdaBoost

In [34]:
model_ada = AdaBoostClassifier()
start = time()
model_ada.fit(x_train,y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_ada = model_ada.predict(x_test)
acc = accuracy_score(pred_ada , y_test)
cm = confusion_matrix(pred_ada , y_test)
cr = classification_report(pred_ada , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

/home/newbie/miniconda3/envs/gpu/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


--------------------------------------------------------
Time taken: 2.2791428565979004 seconds
--------------------------------------------------------
0.9776
--------------------------------------------------------
[[1255   51]
 [   5 1189]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      0.96      0.98      1306
           1       0.96      1.00      0.98      1194

    accuracy                           0.98      2500
   macro avg       0.98      0.98      0.98      2500
weighted avg       0.98      0.98      0.98      2500

--------------------------------------------------------
